# Bachelor Thesis

## LoRa Time on Air

In [103]:
class LoraConf:
    preamble_len = 12
    crc = True
    sf = 7 # [7..12]
    cr = 1 # [0..4]: coding-rate is calculated as follows: 4/(4+cr)
    bw = 125_000
    def __init__(self, preamble_len = 12, crc = True, sf = 7, cr = 1, bw = 125_000):
        self.preamble_len = preamble_len
        self.crc = crc
        self.sf = sf
        self.cr = cr
        self.bw = bw

lora_conf = LoraConf(8, True, 7, 1, 125_000)

Implement time on air (ToA) calculation as described in [STM32WL5X reference manual](https://www.st.com/content/ccc/resource/technical/document/reference_manual/group0/34/ea/c6/91/75/6f/45/27/DM00451556/files/DM00451556.pdf/jcr:content/translations/en.DM00451556.pdf)

In [104]:
def ceil(frac, base):
    return ((frac // base) + 1 ) * base

def lora_time_on_air(lora_conf, user_payload_bytes):
    nb_payload_symbols_frac = ((user_payload_bytes * 8 + int(lora_conf.crc) * 16 - 4 * (lora_conf.sf - 7)) * (4 + lora_conf.cr)) / (4 * lora_conf.sf)
    nb_payload_symbols = ceil(nb_payload_symbols_frac, lora_conf.cr + 4)
    return (lora_conf.preamble_len + nb_payload_symbols + 4.25 + 8) * (2**lora_conf.sf / lora_conf.bw)

### ToA calculations

#### Ping_t
```c
typedef struct {
    uint8_t device_id;
    uint_t packet_id;
} Ping_t;
```
#### AnchorResponse_t
```c
typedef struct {
    char anchor_id;
    int16_t recv_rssi;
} AnchorResponse_t;
```
#### Ack_t
```c
typedef struct {
    char anchor_id;
    uint8_t packet_id;
} Ack_t;
```

In [105]:
print(f"ToA(Ping_t) = {lora_time_on_air(lora_conf, 2) * 1000}ms")
print(f"ToA(AnchorResponse_t) = {lora_time_on_air(lora_conf, 3) * 1000}ms")
print(f"ToA(Ack_t) = {lora_time_on_air(lora_conf, 2) * 1000}ms")

ToA(Ping_t) = 30.976ms
ToA(AnchorResponse_t) = 30.976ms
ToA(Ack_t) = 30.976ms


## Battery Lifetime

### Device configuration

In [106]:
cr2 = 850 #mAh
little_lipo = 3500 #mAh
big_lipo = 40000 #mAh
tx_power_draw = 20 #mA
rx_power_draw = 8.9 #mA
idle_power_draw = 0.06 #mA
tx_time = 40 #ms
rx_time = 200 #ms

In [107]:
def power_draw(tx_power, rx_power, idle_power, t_tx, t_rx):
    return (tx_power * (t_tx / 1000)) + (rx_power * (t_rx / 1000)) + (idle_power * ((1000 - t_rx - t_rx) / 1000))

# capacity in mAh, power_draw in mA
def calc_bat_time(capacity, power_draw):
    return capacity / power_draw / 24

In [108]:
datasheet_power_draw = power_draw(tx_power=23.5, rx_power=8.9, idle_power=0.4, t_tx=40, t_rx=200)
print(f"cr2 = {calc_bat_time(cr2, datasheet_power_draw)} days")
print(f"little lipo = {calc_bat_time(little_lipo, datasheet_power_draw)} days")
print(f"big lipo = {calc_bat_time(big_lipo, datasheet_power_draw)} days")

cr2 = 11.965090090090092 days
little lipo = 49.26801801801802 days
big lipo = 563.063063063063 days


## Helper functions

### Convert MCU timestamp to Python datetime

In [109]:
from datetime import datetime, timedelta

def convert_timestamp(ts_str: str) -> datetime:
    # Remove the trailing 's' and split into seconds and milliseconds
    seconds, milliseconds = ts_str.split('s')
    # Create a datetime object
    return datetime.min + timedelta(seconds=int(seconds), milliseconds=int(milliseconds))

### Trilateration function

In [110]:
import numpy as np

def trilaterate(anchor_positions, distances):
    """
    Estimate the position of a point using trilateration.
    
    :param anchor_positions: List of (x, y, z) coordinates of the anchors
    :param distances: List of distances from each anchor to the point
    :return: Estimated (x, y, z) position of the point
    """
    # Ensure we have exactly 3 anchor positions and distances
    if len(anchor_positions) != 3 or len(distances) != 3:
        raise ValueError("Trilateration requires exactly 3 anchor positions and distances")
    
    # Extract anchor positions
    P1, P2, P3 = np.array(anchor_positions)
    
    # Calculate the vectors between anchors
    P21 = P2 - P1
    P31 = P3 - P1
    
    # Create the matrix A and vector b for the equation Ax = b
    A = np.array([
        [2*P21[0], 2*P21[1]],
        [2*P31[0], 2*P31[1]]
    ])
    
    b = np.array([
        distances[0]**2 - distances[1]**2 + np.dot(P2, P2) - np.dot(P1, P1),
        distances[0]**2 - distances[2]**2 + np.dot(P3, P3) - np.dot(P1, P1)
    ])
    
    # Solve the system of equations
    try:
        x = np.linalg.lstsq(A, b, rcond=None)[0]
    except np.linalg.LinAlgError:
        raise ValueError("Unable to solve the trilateration problem. The anchor positions may be collinear.")
    
    # The solution is relative to P1, so add P1 to get the final position
    estimated_position = P1 + x
    
    return estimated_position

## Evaluation

In [111]:
from typing import List, Union, Dict
import pandas as pd
from dataclasses import dataclass
from datetime import datetime

### Distance estimation

In [112]:
@dataclass
class DistMsg:
    experiment: str
    distance: float
    timestamp: datetime
    anchor_id: int
    recv_rssi: int

In [113]:
import pandas as pd
from pathlib import Path
import re

def find_data_start(file_path):
    """
    Find the line number where the CSV data actually starts by looking for
    the first line that starts with a number
    
    Parameters:
    file_path (Path): Path to the CSV file
    
    Returns:
    int: Line number where data starts
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data_start = 0
        while True:
            line = f.readline()
            if not line:
                break;
            elif re.match(r"^\d+", line):
                break;
            else:
                data_start += 1
        f.close()
        return data_start

def parse_distance_csv(dir_path: str) -> pd.DataFrame:
    """
    Parse all CSV files in a specified directory (non-recursive).
    Adds directory information to the resulting DataFrame.
    
    Parameters:
    directory (str): Directory path containing CSV files
    
    Returns:
    pandas.DataFrame: Combined DataFrame with all CSV data and directory information
    """
    # Convert directory to Path object for better path handling
    dir_path = Path(dir_path)
    
    # List to store all DataFrames
    dfs = []
    
    # Get all CSV files in the directory (non-recursive)
    csv_files = [f for f in dir_path.iterdir() if f.is_file() and f.suffix.lower() == '.csv']
    
    for file_path in csv_files:
        try:
            data_start = find_data_start(file_path)

            # Read CSV file
            df = pd.read_csv(file_path, header=None, index_col=False, skiprows=data_start, names=['timestamp', 'anchor_id', 'recv_rssi_dbm'])

            
            # Remove trailing colon from timestamp and convert to datetime
            df['timestamp'] = df['timestamp'].str.split(":").str[0].apply(convert_timestamp)
            df['anchor_id'] = (df['anchor_id']
                            .apply(str)
                            .str.replace(r"^1$", "A", regex=True)
                            .str.replace(r"^2$", "B", regex=True)
                            .str.replace(r"^3$", "C", regex=True))
            
            # Add file and directory information
            dist = file_path.name.rstrip(".csv").strip()
            if re.match(r"^\d+cm$", dist) != None:
                df['distance_m'] = float(dist.rstrip("cm")) * 0.01
            elif re.match(r"^\d+m$", dist) != None:
                df['distance_m'] = float(dist.rstrip("m"))
            else:
                df['distance_m'] = 0.0
                
            df['experiment'] = str(dir_path.name).lstrip("experiment")
            
            dfs.append(df)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {str(e)}")
            continue
    
    # Combine all DataFrames
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()  # Return empty DataFrame if no CSVs found

def df_to_dist_msgs(df: pd.DataFrame) -> List[DistMsg]:
    messages = []
    for _, row in df.iterrows():
        messages.append(DistMsg(row['experiment'], row['distance_m'], row['timestamp'], row['anchor_id'], row['recv_rssi_dbm']))
    return messages

def print_dist_msgs(messages: List[DistMsg]):
    for msg in messages:
        print(f"DistMsg: Experiment={msg.experiment}, Distance={msg.distance}, Timestamp={msg.timestamp}, Anchor ID={msg.anchor_id}, RSSI={msg.recv_rssi}")


In [117]:
experiments = ['01_1', '01_2', '02_1', '02_2']
dist_df = pd.concat([parse_distance_csv(f"../../data/experiment{e}") for e in experiments], ignore_index=True)

MIN_DISTANCE_M = 1.0
dist_df = dist_df[dist_df['distance_m'] >= MIN_DISTANCE_M]
dist_msgs = df_to_dist_msgs(dist_df)
print_dist_msgs(dist_msgs)


DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:53.365000, Anchor ID=A, RSSI=-31
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:53.764000, Anchor ID=A, RSSI=-31
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:54.164000, Anchor ID=A, RSSI=-31
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:54.563000, Anchor ID=A, RSSI=-32
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:54.962000, Anchor ID=A, RSSI=-32
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:55.362000, Anchor ID=A, RSSI=-31
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:55.761000, Anchor ID=A, RSSI=-32
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:56.161000, Anchor ID=A, RSSI=-31
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:56.560000, Anchor ID=A, RSSI=-32
DistMsg: Experiment=01_1, Distance=5.0, Timestamp=0001-01-01 00:00:56.959000, Anchor ID=A, RSSI=-32


### Calculate distance from RSSI with log-normal path-loss model

In [63]:
import math

curves = {
    'exp01_1': [-18.636596325051777, -19.39053544037741],
    'exp01_2': [-20.905823053900388, -20.905823053900388],
    'exp01_2_10': [-23.923402226978908, -11.24909610872239],
    'exp02_1': [-21.341727735973773,  -12.79214877723544],
    'exp02_2': [-17.550701777837332, -17.64915028968665],
}

def rssi_to_distance(rssi: int, experiment: str):
    return 10**((rssi-curves[experiment][1])/curves[experiment][0])

### Localization

#### Datatypes for data parsing

In [10]:
@dataclass
class PingMessage:
    timestamp: datetime
    direction: str
    device_id: str
    packet_id: int

@dataclass
class AnchorResponseMessage:
    timestamp: datetime
    direction: str
    anchor_id: str
    packet_id: int
    recv_rssi: int

@dataclass
class AckMessage:
    timestamp: datetime
    direction: str
    receiver_id: str
    packet_id: int

PosMsg = Union[PingMessage, AnchorResponseMessage, AckMessage]

#### Parse data from CSV

In [13]:
def parse_position_csv(file_path: str) -> pd.DataFrame:
    # Read CSV file using pandas
    df = pd.read_csv(file_path, header=None, index_col=False, names=['timestamp', 'direction', 'message_type', 'id', 'packet_id', 'recv_rssi'])

    
    # Remove trailing colon from timestamp and convert to datetime
    df['timestamp'] = df['timestamp'].str[:-1].apply(convert_timestamp)
    df['direction'] = df['direction'].str.strip()
    df['message_type'] = df['message_type'].str.strip()
    df['id'] = df['id'].str.strip()
    return df
    
def df_to_pos_msgs(df: pd.DataFrame):
    messages = []
    
    for _, row in df.iterrows():
        if row['message_type'] == 'Ping':
            messages.append(PingMessage(row['timestamp'], row['direction'], row['id'], int(row['packet_id'])))
        elif row['message_type'] == 'AnchorResponse':
            messages.append(AnchorResponseMessage(row['timestamp'], row['direction'], row['id'], int(row['packet_id']), int(row['recv_rssi'])))
        elif row['message_type'] == 'Ack':
            messages.append(AckMessage(row['timestamp'], row['direction'], row['id'], int(row['packet_id'])))

    return messages

def print_pos_msgs(messages: List[PosMsg]):
    for msg in messages:
        if isinstance(msg, PingMessage):
            print(f"Ping: Timestamp={msg.timestamp}, Direction={msg.direction}, Device ID={msg.device_id}, Packet ID={msg.packet_id}")
        elif isinstance(msg, AnchorResponseMessage):
            print(f"AnchorResponse: Timestamp={msg.timestamp}, Direction={msg.direction}, Anchor ID={msg.anchor_id}, Packet ID={msg.packet_id}, RSSI={msg.recv_rssi}")
        elif isinstance(msg, AckMessage):
            print(f"Ack: Timestamp={msg.timestamp}, Direction={msg.direction}, Receiver ID={msg.receiver_id}, Packet ID={msg.packet_id}")

def analyze_pos_df(df):
    print("Data Summary:")
    print(df.describe())
    
    print("\nMessage Type Distribution:")
    print(df['message_type'].value_counts())
    
    print("\nDirection Distribution:")
    print(df['direction'].value_counts())
    
    if 'recv_rssi' in df.columns:
        print("\nRSSI Statistics:")
        print(df['recv_rssi'].describe())
    
    print("\nTimestamp Range:")
    print(f"Start: {df['timestamp'].min()}")
    print(f"End: {df['timestamp'].max()}")
    print(f"Duration: {df['timestamp'].max() - df['timestamp'].min()}")

def group_anchor_responses(messages: List[PosMsg]) -> Dict[int, List[AnchorResponseMessage]]:
    anchor_responses = {}
    for msg in messages:
        if isinstance(msg, AnchorResponseMessage) and msg.direction == 'RX':
            if msg.packet_id not in anchor_responses:
                anchor_responses[msg.packet_id] = []
            anchor_responses[msg.packet_id].append(msg)
    return anchor_responses

def print_grouped_responses(grouped_responses: Dict[int, List[AnchorResponseMessage]]):
    for packet_id, responses in grouped_responses.items():
        print(f"\nPacket ID: {packet_id}")
        for response in responses:
            print(f"  Anchor ID: {response.anchor_id}, RSSI: {response.recv_rssi}, Timestamp: {response.timestamp}")

file_path = "../../data/experiment06/pos3.csv"  # Replace with the actual path to your CSV file
df = parse_position_csv(file_path)
messages = df_to_pos_msgs(df)
a_res = group_anchor_responses(messages)
print_grouped_responses(a_res)

print("\nAdditional Data Analysis:")
analyze_pos_df(df)


Packet ID: 2
  Anchor ID: B, RSSI: -63, Timestamp: 0001-01-01 00:00:02.092000
  Anchor ID: C, RSSI: -73, Timestamp: 0001-01-01 00:00:02.157000
  Anchor ID: A, RSSI: -76, Timestamp: 0001-01-01 00:00:02.222000

Packet ID: 3
  Anchor ID: B, RSSI: -62, Timestamp: 0001-01-01 00:00:03.092000
  Anchor ID: A, RSSI: -75, Timestamp: 0001-01-01 00:00:03.157000
  Anchor ID: C, RSSI: -79, Timestamp: 0001-01-01 00:00:03.222000

Packet ID: 4
  Anchor ID: B, RSSI: -62, Timestamp: 0001-01-01 00:00:04.092000
  Anchor ID: C, RSSI: -73, Timestamp: 0001-01-01 00:00:04.157000
  Anchor ID: A, RSSI: -78, Timestamp: 0001-01-01 00:00:04.222000

Packet ID: 5
  Anchor ID: B, RSSI: -62, Timestamp: 0001-01-01 00:00:05.092000
  Anchor ID: C, RSSI: -73, Timestamp: 0001-01-01 00:00:05.157000
  Anchor ID: A, RSSI: -82, Timestamp: 0001-01-01 00:00:05.222000

Packet ID: 6
  Anchor ID: B, RSSI: -61, Timestamp: 0001-01-01 00:00:06.092000
  Anchor ID: C, RSSI: -72, Timestamp: 0001-01-01 00:00:06.157000
  Anchor ID: A, RSSI

### Calculate position based on RSSI measurements

In [68]:
anchor_a = np.array((1376568.536031973, 6683153.2447312595 ))
anchor_b = np.array((1376485.2164853222, 6683448.359372672 ))
anchor_c = np.array((1376154.6749028682, 6683475.401849588 ))

anchor_positions = [anchor_a, anchor_b, anchor_c];

for packet_id in a_res:
    distances = list(map(lambda r: rssi_to_distance(r.recv_rssi, 'exp01_2_10'), a_res[packet_id]))
    print(distances)

try:
    estimated_position = trilaterate(anchor_positions, list(distances))
    print(f"Estimated position: {estimated_position}")
except ValueError as e:
    print(f"Error: {e}")

[145.61014960800816, 381.2346752019708, 508.8533278426195]
[132.24874926033996, 462.1602020553594, 679.1924399828836]
[132.24874926033996, 381.2346752019708, 616.868748068884]
[132.24874926033996, 381.2346752019708, 906.5527241134141]
[120.11341055556727, 346.2520236800834, 747.8128402095532]
[120.11341055556727, 560.2639693010451, 823.3661199120107]
[99.08122087731725, 462.1602020553594, 560.2639693010451]
[120.11341055556727, 381.2346752019708, 616.868748068884]
[132.24874926033996, 462.1602020553594, 747.8128402095532]
[132.24874926033996, 381.2346752019708, 679.1924399828836]
[132.24874926033996, 346.2520236800834, 616.868748068884]
[120.11341055556727, 346.2520236800834, 679.1924399828836]
[120.11341055556727, 314.47943143953927, 616.868748068884]
[145.61014960800816, 314.47943143953927, 616.868748068884]
[132.24874926033996, 346.2520236800834, 616.868748068884]
[120.11341055556727, 419.75170579981227, 616.868748068884]
[132.24874926033996, 381.2346752019708, 616.868748068884]
[13